In [7]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks

In [8]:
TRAIN_DIR = './datasets/images/train' 
VAL_DIR   = './datasets/images/validation'
IMG_SIZE = 48
BATCH_SIZE = 64
MODEL_FILENAME = 'emotion_model.h5'

In [4]:
# If you see "Import could not be resolved" or ModuleNotFoundError, install required packages:
# (Run this in the Jupyter cell; uses the magic %pip install)
%pip install -q tensorflow tensorflowjs keras keras-preprocessing

import os
import tensorflow as tf
# Use standalone keras preprocessing import to avoid unresolved import diagnostics in some editors
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks
import tensorflowjs as tfjs

TRAIN_DIR = './datasets/images/train' 
VAL_DIR   = './datasets/images/validation'
OUTPUT_DIR = '../public/models/web_model_emotion'
IMG_SIZE = 48
BATCH_SIZE = 64

def train_emotion_model():
    print(f"Checking GPU: {len(tf.config.list_physical_devices('GPU')) > 0}")

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        zoom_range=0.15,
        horizontal_flip=True
    )

    val_datagen = ImageDataGenerator(rescale=1./255)

    print("Loading Training Data...")
    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        color_mode='grayscale'
    )

    print("Loading Validation Data...")
    validation_generator = val_datagen.flow_from_directory(
        VAL_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        color_mode='grayscale'
    )

    # 2. Define Model Architecture (CNN)
    model = models.Sequential([
        # Convolutional Block 1
        layers.Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.25),

        # Convolutional Block 2
        layers.Conv2D(128, (5,5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.25),

        # Convolutional Block 3
        layers.Conv2D(512, (3,3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.25),

        # Dense Layers (The Decision Maker)
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(7, activation='softmax') # 7 Emotions (angry, disgust, fear, happy, neutral, sad, surprise)
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 3. Train
    print("Starting Training...")
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    model.fit(
        train_generator,
        epochs=30,
        validation_data=validation_generator,
        callbacks=[early_stopping]
    )

    # 4. Save & Convert
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        
    print("Saving Model...")
    model.save('emotion_model.h5')
    
    print("Converting for Web...")
    tfjs.converters.save_keras_model(model, OUTPUT_DIR)
    print(f"✅ Model saved to {OUTPUT_DIR}")

if __name__ == "__main__":
    train_emotion_model()

error: resolution-too-deep

× Dependency resolution exceeded maximum depth
╰─> Pip cannot resolve the current dependencies as the dependency graph is too complex for pip to solve efficiently.

hint: Try adding lower bounds to constrain your dependencies, for example: 'package>=2.0.0' instead of just 'package'.

Link: https://pip.pypa.io/en/stable/topics/dependency-resolution/#handling-resolution-too-deep-errors


Note: you may need to restart the kernel to use updated packages.


ImportError: DLL load failed while importing _pywrap_tf2: The specified module could not be found.